## Podstawy podstaw analizy danych EEG w MNE-Python

### Materiały do przejrzenia, dotyczące kroków przygotowania danych (ang. *preprocessing*):  

https://sccn.ucsd.edu/wiki/Makoto%27s_preprocessing_pipeline  

https://www.frontiersin.org/articles/10.3389/fnins.2018.00309/full

https://www.frontiersin.org/articles/10.3389/fnins.2013.00267/full

https://www.biorxiv.org/content/early/2017/12/28/240044

Na samym początku warto obejrzeć to krótkie wystąpienie konferencyjne, które dobrze podsumowuje ogólną ideę pracy z MNE.  
[Wystąpienie Alexandra Gramforta](https://www.youtube.com/watch?v=iW0g_KDYIYg)

### Wybór środowiska graficznego:  
https://matplotlib.org/faq/usage_faq.html#what-is-a-backend  
(będzie to miało znaczenie dla sposobu wyświetlania wykresów - wybór poniżej pozwoli wyświetlić je w osobnym oknie, ale przez to będą interaktywne)

In [1]:
import matplotlib
matplotlib.use('Qt5Agg')

Oprócz tego importujemy biblioteki niezbędne do pracy z danymi:  
**numpy** do operacji matematycznych  
**pandas** do działania na bazach danych  
**maplotlib** do generowania wykresów  
**mne** do analizy EEG

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne

### Wczytywanie danych

Moduł IO w MNE służy do wczytywania danych MEEG pochodzących z różnych źródeł i różnych systemów danych. My zbieraliśmy dane do formatu **.bdf** ([BioSemi Data Format](https://www.biosemi.com/faq/file_format.htm)).  
Jest to jednak po prostu zmodyfikowana wersja pliku **.edf** (*European Data Format*), więc wczytywanie wszelkich pochodnych tego standardu odbywa się za pomoca jednej funkcji:

In [3]:
data = mne.io.read_raw_edf('./0727 resting otwarte.bdf', preload=True, montage='biosemi64');

Extracting EDF parameters from ./0727 resting otwarte.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']. Their position has been left untouched.
Reading 0 ... 1515519  =      0.000 ...   740.000 secs...


<ipython-input-3-533762afd7d1>:1: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']. Their position has been left untouched.
  data = mne.io.read_raw_edf('./0727 resting otwarte.bdf', preload=True, montage='biosemi64');


### Podgląd surowych danych

Pod przyciskiem "Help" znajdziecie legendę skrótów klawiszowych, które pozwolą Wam manipulować wyświetlaniem danych.

In [6]:
data.plot();

In [5]:
data.info

<Info | 18 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, ...
    chs : list | 80 items (EEG: 79, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 67 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 417.0 Hz
    meas_date : int | 1533810697
    nchan : int | 80
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 2048.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info

In [6]:
print(data.ch_names)

['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'STI 014']


In [71]:
data.plot_psd();

Effective window size : 1.000 (s)


### Wczytanie markerów wydarzeń w zapisie EEG (tzw. events).

Kanał STI 014 zawiera zakodowane informacje o triggerach, które procedura eksperymentalna wysyłała do wzmacniacza EEG, wskazując co w danym momencie dzieje się z osobą badaną.  
Nasz plik powinien zawierać wyłącznie dwa triggery - na początek i na koniec pomiaru stanu spoczynkowego, tym niemniej spróbujemy je wydobyć.

In [7]:
x = mne.find_events(data)

Trigger channel has a non-zero initial value of 130816 (consider using initial_event=True to detect this event)
3 events found
Event IDs: [ 65291  65299 131060]


In [9]:
x[:, 2] -= x[:, 1]
x[:, 1] -= x[:, 1]

In [10]:
x

array([[   1977,       0,     244],
       [   2388,       0,      11],
       [1231996,       0,      19]], dtype=int64)

### Przycinanie zapisu na podstawie triggerów.

In [11]:
data.crop(tmin=x[1,0]/2048,tmax=x[2,0]/2048)

<RawEDF  |  0727 resting otwarte.bdf, n_channels x n_times : 80 x 1229609 (600.4 sec), ~750.7 MB, data loaded>

### Wybór interesujących nas elektrod.

In [12]:
EEG_chans = ['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 
                    'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 
                    'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']
data.pick_channels(EEG_chans)

<RawEDF  |  0727 resting otwarte.bdf, n_channels x n_times : 64 x 1229609 (600.4 sec), ~600.6 MB, data loaded>

### Wybór elektrody referencyjnej

W zasadzie dowolna elektroda (oraz dowolna ich kombinacja) może stać się punktem odniesienia dla sygnału z pozostałych.  
Poniżej wykorzystujemy bardzo popularną referencję uśrednioną - wirtualny kanał będący zwykłą średnią ze wszystkich elektrod na skalpie.

In [13]:
data.set_eeg_reference();

Applying average reference.
Applying a custom EEG reference.


In [14]:
data.plot();

Poniżej te same dane odniesione do średniej z dwóch elektrod na linii środkowej.

### Downsampling

In [15]:
data.resample(256)

<RawEDF  |  0727 resting otwarte.bdf, n_channels x n_times : 64 x 153701 (600.4 sec), ~75.2 MB, data loaded>

### Filtrowanie

In [16]:
data.filter(0.01,None)
data.filter(None,100)

Setting up high-pass filter at 0.01 Hz
l_trans_bandwidth chosen to be 0.0 Hz
Filter length of 84481 samples (330.004 sec) selected
Setting up low-pass filter at 1e+02 Hz
h_trans_bandwidth chosen to be 25.0 Hz
Filter length of 35 samples (0.137 sec) selected


<RawEDF  |  0727 resting otwarte.bdf, n_channels x n_times : 64 x 153701 (600.4 sec), ~75.2 MB, data loaded>

In [17]:
data.plot_psd();

Effective window size : 8.000 (s)


### Korekcja artefaktów - ICA

In [18]:
from mne.preprocessing import ICA

In [20]:
data_to_ica = data.copy()

In [21]:
data_to_ica.filter(1,None)

Setting up high-pass filter at 1 Hz
l_trans_bandwidth chosen to be 1.0 Hz
Filter length of 845 samples (3.301 sec) selected


<RawEDF  |  0727 resting otwarte.bdf, n_channels x n_times : 64 x 153701 (600.4 sec), ~75.2 MB, data loaded>

In [22]:
ica_data = ICA(method='fastica', random_state=0, max_iter=500)
ica_data.fit(data_to_ica)

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 64
Fitting ICA took 33.1s.


<ICA  |  raw data decomposition, fit (fastica): 153701 samples, 64 components, channels used: "eeg">

In [24]:
ica_data.plot_components(range(20), inst=data_to_ica)

<Figure size 750x700 with 20 Axes>

In [25]:
ica_data.exclude = [0, 2]

In [26]:
ica_data.apply(data)

Transforming to ICA space (64 components)
Zeroing out 2 ICA components


<RawEDF  |  0727 resting otwarte.bdf, n_channels x n_times : 64 x 153701 (600.4 sec), ~75.2 MB, data loaded>

In [27]:
data.plot()

<Figure size 2560x1438 with 4 Axes>

In [23]:
ica_data.plot_properties(data)

    using multitaper spectrum estimation with 7 DPSS windows


C:\Users\marcin\AppData\Local\conda\conda\envs\mne\lib\site-packages\mkl_fft\_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


KeyboardInterrupt: 

In [24]:
ica_data.get_sources(data).plot()

<Figure size 3840x2056 with 4 Axes>

https://labeling.ucsd.edu/labelfeedback